In [1]:
import streamlit as st
import yfinance as yf
import pandas as pd
import schedule
import time
import ollama
from datetime import datetime, timedelta


In [3]:
stock = yf.Ticker("AAPL")
dow_jones = yf.Ticker("^DJI")
data = stock.history(period="1d", interval="1m")
dow_data = dow_jones.history(period="1d", interval="1m")
dow_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-11-27 09:30:00-05:00,44837.750000,44908.941406,44837.750000,44898.531250,0,0.0,0.0
2024-11-27 09:31:00-05:00,44894.148438,44910.238281,44877.648438,44905.921875,3703622,0.0,0.0
2024-11-27 09:32:00-05:00,44906.281250,44952.460938,44906.281250,44952.460938,2218529,0.0,0.0
2024-11-27 09:33:00-05:00,44949.269531,44995.738281,44945.250000,44995.738281,2251201,0.0,0.0
2024-11-27 09:34:00-05:00,44993.820312,45003.058594,44977.121094,44989.800781,2579354,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-27 15:55:00-05:00,44725.140625,44732.710938,44722.859375,44730.878906,2568069,0.0,0.0
2024-11-27 15:56:00-05:00,44732.871094,44733.371094,44722.628906,44724.738281,2085134,0.0,0.0
2024-11-27 15:57:00-05:00,44724.500000,44728.460938,44713.980469,44719.351562,2711174,0.0,0.0


In [4]:
rolling_window = pd.DataFrame()
dow_rolling_window = pd.DataFrame()
daily_high = float('-inf')
daily_low = float('-inf')
buying_momentum = 0
selling_momentum = 0

def process_stock_update():
    global rolling_window, data, dow_rolling_window, dow_data
    global daily_high, daily_low, buying_momentum, selling_momentum

    if not data.empty and not dow_data.empty:
        update = data.iloc[0].to_frame().T 
        dow_update = dow_data.iloc[0].to_frame().T 
        data = data.iloc[1:] 
        dow_data = dow_data.iloc[1:] 

        rolling_window = pd.concat([rolling_window, update], ignore_index=False)
        dow_rolling_window = pd.concat([dow_rolling_window, dow_update], ignore_index=False)

        daily_high = max(daily_high, update['Close'].values[0])
        daily_low = min(daily_low, update['Close'].values[0])

        if len(rolling_window) >= 2:
            price_change = update['Close'].values[0] - rolling_window['Close'].iloc[-2]
            if price_change > 0:
                buying_momentum += price_change
            else:
                selling_momentum += abs(price_change)

# Analyzing the Stock Trends
**Exponential Moving Average** : Puts more weight on recent prices to identify short-term trends.

**Relative Strength Index (RSI)**: Measures price movement speed and oscillates between 0 and 100 to identify overbought / oversold conditions.

**Bollinger Bands** : Help assess market volatility with upper and lower brands around the moving average.

In [ ]:
def calculate_insights(window, dow_window):
    if len(window) >= 5:
        rolling_avg = window['Close'].rolling(window=5).mean().iloc[-1]
        ema = window['Close'].ewm(span=5, adjust=False).mean().iloc[-1]
        std = window['Close'].rolling(window=5).std().iloc[-1]
        bollinger_upper = rolling_avg + (2 * std)
        bollinger_lower = rolling_avg - (2 * std)

        delta = window['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=14, min_periods=1).mean().iloc[-1]
        avg_loss = loss.rolling(window=14, min_periods=1).mean().iloc[-1]
        rs = avg_gain /  avg_loss if avg_loss != 0 else float('nan')
        rsi = 100 - (100 / (1 + rs))

        dow_rolling_avg = dow_window['Close'].rolling(window=5).mean().iloc[-1]

        market_open_duration = get_market_open_duration(window)
        if int(market_open_duration) % 5 == 0:
            get_natu
        